## 네이버영화 감성분석

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/swkim01/DSAC4/blob/main/gg-62-네이버영화감성분석_신경망.ipynb"><img src="https://github.com/swkim01/DSAC4/raw/main/colab_logo_32px.png" />구글 코랩에서 실행</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/swkim01/DSAC4/blob/main/gg-62-네이버영화감성분석_신경망.ipynb"><img src="https://github.com/swkim01/DSAC4/raw/main/GitHub-Mark-32px.png" />깃헙에서 소스 보기</a>
  </td>
</table>

In [ ]:
# GPU 런타임 설정을 한 후 실행한다.
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 448 kB 46.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [ ]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0   1727      0 --:--:-- --:--:-- --:--:--  1727
100   148    0   148    0     0    414      0 --:--:-- --:--:-- --:--:--   414
100   318  100   318    0     0    482      0 --:--:-- --:--:-- --:--:--     0
100 14.0M  100 14.0M    0     0  10.3M      0  0:00:01  0:00:01 --:--:-- 10.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0   1841      0 --:--:-- --:--:-- --:--:--  1841
100   147    0   147    0     0    500      0 --:--:-- --:--:-- --:--:-- 24500
100   318  100   318    0     0    528      0 --:--:-- --:--:-- --:--:--   528
100 4827k  100 4827k    0     0  4956k      0 --:--:-- --:--:-- --:--:-- 4956k


In [ ]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [ ]:
df_train[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()

twitter_tag = Twitter()


/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, 
                     max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.fit_transform(text_test)

In [ ]:
X_train.shape, y_train.shape

((150000, 3000), (150000,))

In [ ]:
X_test.shape, y_test.shape

((50000, 3000), (50000,))

In [ ]:
y_test[:3]

array([1, 0, 0])

# 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.8406266666666666
Testing :  0.82616


# MLP

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model.fit(X_train, y_train_cat,
        epochs=nb_epoch, 
        batch_size=batch_size) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
150000/150000 [==============================] - 9s 59us/step - loss: 0.3977 - acc: 0.8137
Epoch 2/5
150000/150000 [==============================] - 7s 47us/step - loss: 0.3507 - acc: 0.8423
Epoch 3/5
150000/150000 [==============================] - 7s 47us/step - loss: 0.3256 - acc: 0.8569
Epoch 4/5
150000/150000 [==============================] - 7s 47us/step - loss: 0.2940 - acc: 0.8745
Epoch 5/5
150000/150000 [==============================] - 7s 47us/step - loss: 0.2576 - acc: 0.8922


In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.8289

# RNN

In [ ]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)

(150000, 1, 3000)


In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(128, input_shape=(X_train_rnn.shape[1], 
              X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [ ]:
model.fit(X_train_rnn, y_train_cat,
          batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
150000/150000 [==============================] - 11s 72us/step - loss: 0.3938 - acc: 0.8174
Epoch 2/5
150000/150000 [==============================] - 10s 68us/step - loss: 0.3429 - acc: 0.8461
Epoch 3/5
150000/150000 [==============================] - 10s 68us/step - loss: 0.3056 - acc: 0.8685
Epoch 4/5
150000/150000 [==============================] - 10s 68us/step - loss: 0.2535 - acc: 0.8932
Epoch 5/5
150000/150000 [==============================] - 10s 68us/step - loss: 0.1975 - acc: 0.9191


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.82738

# LSTM

In [ ]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rnn, y_train_cat,
          batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
150000/150000 [==============================] - 19s 123us/step - loss: 0.3988 - acc: 0.8149
Epoch 2/5
150000/150000 [==============================] - 17s 115us/step - loss: 0.3564 - acc: 0.8364
Epoch 3/5
150000/150000 [==============================] - 17s 115us/step - loss: 0.3385 - acc: 0.8473
Epoch 4/5
150000/150000 [==============================] - 17s 114us/step - loss: 0.3128 - acc: 0.8625
Epoch 5/5
150000/150000 [==============================] - 17s 114us/step - loss: 0.2782 - acc: 0.8815


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
metrics.accuracy_score(y_test, y_pred)

0.8339